In [48]:
import contextlib
import os
from time import sleep

import qibo
from qibo import gates
from qibo.models.circuit import Circuit

from qiboconnection.api import API

In [ ]:
import qililab as ql

drag_pulse = ql.IQPair.DRAG(amplitude=1.0, duration=40, num_sigmas=4.5, drag_coefficient=-2.0)
readout_pulse = ql.IQPair(I=ql.Square(amplitude=1.0, duration=2000), Q=ql.Square(amplitude=0.0, duration=2000))
weights = ql.IQPair(I=ql.Square(amplitude=1.0, duration=2000), Q=ql.Square(amplitude=1.0, duration=2000))
time_of_flight = 40
shots = 1000

qp = ql.QProgram()
gain = qp.variable(domain=ql.Domain.Voltage)
with qp.average(shots=shots):
    with qp.for_loop(variable=gain, start=0.0, stop=1.0, step=0.1):
        qp.set_gain(bus="drive_q0_bus", gain=gain)
        qp.play(bus="drive_q0_bus", waveform=drag_pulse)
        qp.sync()
        qp.play(bus="readout_q0_bus", waveform=readout_pulse, wait_time=time_of_flight)
        qp.acquire(bus="readout_q0_bus", weights=weights)
        qp.wait(bus="readout_q0_bus", duration=10000)

## Connect to the Qibo Service API

In [49]:
qibo_api = API.login(username="ablanco", api_key="8a19dc8f-793d-4aa8-8ee5-03ec6e8152fb")

In [ ]:
os.environ["QUANTUM_SERVICE_URL"] = "https://dev-api.qaas.qilimanjaro.tech"
os.environ["QIBOCONNECTION_ENVIRONMENT"] = "development"
qibo_api = API.login(username="qili-admin-test", api_key="8b2433a2-365c-4dd6-aa02-646ea42ba950")

In [10]:
qibo_api.get_job(8696).status

Your job with id 8696 is still pending. Job queue position: 2


'queued'

In [11]:
job_id = qibo_api.execute(qprogram=qp.to_dict(), device_ids=[9])

NameError: name 'qp' is not defined

In [ ]:
ql.QProgram.from_dict(qibo_api.get_job(job_id[0] - 1).description["attributes"])

In [22]:
qibo_api.get_job(8689).result

Your job with id 8689 is completed.


[{'probabilities': {'00': 0.75125,
   '01': 0.144375,
   '10': 0.0345,
   '11': 0.069875}}]

## List all jobs metadata: list_jobs()

In [ ]:
jobs_df = qibo_api.list_jobs().dataframe
jobs_df.head()

In [ ]:
jobs_df[(jobs_df["job_type"] == "qprogram") & (jobs_df["user_id"] == 33)]

In [ ]:
qibo_api.get_job(8204).result.to_dict()

## Get specific job data: get_job()

In [ ]:
# get experiment
experiment_data = qibo_api.get_job(job_id=10320)
experiment_data

In [ ]:
experiment_data.description

In [ ]:
# get circuit
circuit_data = qibo_api.get_job(job_id=10323)
circuit_data

In [ ]:
circuit_data.description.draw()

## Load an already existing connection to the Qilimanjaro Service API

In [ ]:
qibo_api = API()

## Check a connection is alive

In [ ]:
qibo_api.ping()

## List all registered devices

In [ ]:
devices = qibo_api.list_devices()
devices

## Selecting a specific device

In [50]:
qibo_api.select_device_id(device_id=9)

## Creating an experiment to be executed on a remote device

In [ ]:
qibo.set_backend("numpy")

In [51]:
circuit = Circuit(1)
circuit.add(gates.H(0))
circuit.add(gates.X(0))
circuit.add(gates.M(0))

MeasurementResult(qubits=(0,), nshots=0)

## Remote execution and getting a job identifier

In [52]:
job_id = qibo_api.execute(circuit=circuit)

In [53]:
qibo_api.cancel_job(job_id[0])

In [54]:
qibo_api.get_job(job_id[0]).status

Your job with id 8734 is cancelled.


'cancelled'

## User retrieves the Qibo result, but it is still pending

In [ ]:
result = qibo_api.get_result(job_id=job_id[0])

## When job is executed remotely, a user can retrieve the actual result

In [ ]:
sleep(1)
result = qibo_api.get_result(job_id=job_id)

In [ ]:
result.state()

In [ ]:
result.frequencies(binary=True, registers=True)

In [ ]:
circuit.draw()

In [ ]:
qibo_api.release_device(device_id=9)

In [ ]:
devices._devices[0].id

## Block and release a remote device to operate on it

In [ ]:
qibo_api.block_device_id(device_id=1)

# Do stuff with the device, knowing no one else will access it

qibo_api.release_device(device_id=1)

## Trying to block a blocked device will raise an exception

In [ ]:
with contextlib.suppress(Exception):
    qibo_api.block_device_id(device_id=1)
    qibo_api.block_device_id(device_id=1)

qibo_api.release_device(device_id=1)

In [ ]:
response = qibo_api.get_job(5685)

In [ ]:
response

response